In [1]:
import os
import zipfile
import gzip

file_path = '/GPUFS/tester01.zip.gz'
output_dir = '/GPUFS'

with gzip.open(file_path, 'rb') as gz_file:
    gz_content = gz_file.read()
    with open(os.path.join(output_dir, 'tester01.zip'), 'wb') as zip_file:
        zip_file.write(gz_content)

with zipfile.ZipFile(os.path.join(output_dir, 'tester01.zip'), 'r') as zip_ref:
    zip_ref.extractall(output_dir)

In [2]:
import os
import zipfile
import gzip

file_path = '/GPUFS/test set.zip.gz'
output_dir = '/GPUFS'

with gzip.open(file_path, 'rb') as gz_file:
    gz_content = gz_file.read()
    with open(os.path.join(output_dir, 'test set.zip'), 'wb') as zip_file:
        zip_file.write(gz_content)

with zipfile.ZipFile(os.path.join(output_dir, 'test set.zip'), 'r') as zip_ref:
    zip_ref.extractall(output_dir)

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, auc
import matplotlib.pyplot as plt

data_path = '/GPUFS/tester01'
cataract_path = os.path.join(data_path, 'cataract')
normal_path = os.path.join(data_path, 'normal')
surgery_path = os.path.join(data_path, 'surgery')

image_paths = []
labels = []

for class_dir, label in [(cataract_path, 0), (normal_path, 1), (surgery_path, 2)]:
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        image_paths.append(img_path)
        labels.append(label)

data_df = pd.DataFrame({'image_path': image_paths, 'label': labels})

image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(0.8, 1.2),
    rescale=1.0/255.0
)

train_df, temp_df = train_test_split(data_df, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_datagen = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw', 
    shuffle=True
)

val_datagen = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw', 
    shuffle=True
)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

history = model.fit(train_datagen, validation_data=val_datagen, epochs=10)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw', 
    shuffle=False
)

y_pred = model.predict(test_generator)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = test_df['label'].values

class_names = ['cataract', 'normal', 'surgery']

print(classification_report(y_true_labels, y_pred_labels, target_names=class_names))

In [5]:
model_save_path = 'DQ.h5'
model.save(model_save_path)